<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Predict outdoor equipment purchase with IBM Watson Machine Learning</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/product-line-prediction/images/products_graphics.png?raw=true" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to get data from the IBM Data Science Experience Community, create a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 2.0 and Apache® Spark 2.0.

You will use a publicly available data set, **GoSales Transactions for Naive Bayes Model**, which details anonymous outdoor equipment purchases. Use the details of this data set to predict clients' interests in terms of product line, such as golf accessories, camping equipment, and others.

## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Deploy a model for online scoring using Wastson Machine Learning API.
-  Score sample scoring data using the Watson Machine Learning API.
-  Explore and visualize prediction result using the plotly package.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create spark ml model](#model)
4.	[Persist model](#persistence)
5.	[Predict locally and visualize](#visualization)
6.	[Deploy and score in a Cloud](#scoring)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered). 
-  Upload **GoSales Transactions** data as a data asset in IBM Data Science Experience.
-  Make sure that you are using a Spark 2.0 kernel.

### Create the GoSales Transactions Data Asset  

The GOSales data is a freely available data set on the Data Science Experience home page.

1.  Go to the [GoSales Transactions for Naive Bayes Model](https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600) data card on the Data Science Experience **Community** page and open the card by double-clicking it.
2.  Click the link icon.
4.  Select the link, copy it by pressing Ctrl+C, and then, click **Close**.
5.  In the following cell, replace the **link_to_data** variable value with the link.

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration.

Load the data to the Spark DataFrame by using *wget* to upload the data to gpfs and then *read* method. 

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.<BR><BR>
!pip install wget --user <BR>

In [40]:
import wget

link_to_data = 'https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600'
filename = wget.download(link_to_data)

print filename

GoSales_Tx_NaiveBayes (4).csv


The csv file GoSales_Tx_NaiveBayes.csv is availble on gpfs now. Load the file to Apache® Spark DataFrame using code below.

In [41]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename)
df_data.take(10)

[Row(PRODUCT_LINE=u'Personal Accessories', GENDER=u'M', AGE=27, MARITAL_STATUS=u'Single', PROFESSION=u'Professional'),
 Row(PRODUCT_LINE=u'Personal Accessories', GENDER=u'F', AGE=39, MARITAL_STATUS=u'Married', PROFESSION=u'Other'),
 Row(PRODUCT_LINE=u'Mountaineering Equipment', GENDER=u'F', AGE=39, MARITAL_STATUS=u'Married', PROFESSION=u'Other'),
 Row(PRODUCT_LINE=u'Personal Accessories', GENDER=u'F', AGE=56, MARITAL_STATUS=u'Unspecified', PROFESSION=u'Hospitality'),
 Row(PRODUCT_LINE=u'Golf Equipment', GENDER=u'M', AGE=45, MARITAL_STATUS=u'Married', PROFESSION=u'Retired'),
 Row(PRODUCT_LINE=u'Golf Equipment', GENDER=u'M', AGE=45, MARITAL_STATUS=u'Married', PROFESSION=u'Retired'),
 Row(PRODUCT_LINE=u'Camping Equipment', GENDER=u'F', AGE=39, MARITAL_STATUS=u'Married', PROFESSION=u'Other'),
 Row(PRODUCT_LINE=u'Camping Equipment', GENDER=u'F', AGE=49, MARITAL_STATUS=u'Married', PROFESSION=u'Other'),
 Row(PRODUCT_LINE=u'Outdoor Protection', GENDER=u'F', AGE=49, MARITAL_STATUS=u'Married', P

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  print top ten records
-  count all records

In [42]:
df_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



As you can see, the data contains five fields. PRODUCT_LINE field is the one we would like to predict (label).

In [43]:
df_data.show()

+--------------------+------+---+--------------+------------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|
+--------------------+------+---+--------------+------------+
|Personal Accessories|     M| 27|        Single|Professional|
|Personal Accessories|     F| 39|       Married|       Other|
|Mountaineering Eq...|     F| 39|       Married|       Other|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|
|      Golf Equipment|     M| 45|       Married|     Retired|
|      Golf Equipment|     M| 45|       Married|     Retired|
|   Camping Equipment|     F| 39|       Married|       Other|
|   Camping Equipment|     F| 49|       Married|       Other|
|  Outdoor Protection|     F| 49|       Married|       Other|
|      Golf Equipment|     M| 47|       Married|     Retired|
|      Golf Equipment|     M| 47|       Married|     Retired|
|Mountaineering Eq...|     M| 21|        Single|      Retail|
|Personal Accessories|     F| 66|       Married|       Other|
|   Camp

In [44]:
df_data.count()

60252

As you can see, the data set contains 60252 records.

<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [45]:
splitted_data = df_data.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print "Number of training records: " + str(train_data.count())
print "Number of testing records : " + str(test_data.count())
print "Number of prediction records : " + str(predict_data.count())

Number of training records: 48176
Number of testing records : 10860
Number of prediction records : 1216


As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [46]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the string fields to numeric ones by using the StringIndexer transformer.

In [47]:
stringIndexer_label = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label").fit(df_data)
stringIndexer_prof = StringIndexer(inputCol="PROFESSION", outputCol="PROFESSION_IX")
stringIndexer_gend = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX")
stringIndexer_mar = StringIndexer(inputCol="MARITAL_STATUS", outputCol="MARITAL_STATUS_IX")

In the following step, create a feature vector by combining all features together.

In [48]:
vectorAssembler_features = VectorAssembler(inputCols=["GENDER_IX", "AGE", "MARITAL_STATUS_IX", "PROFESSION_IX"], outputCol="features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [49]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, indexed labels back to original labels.

In [50]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [51]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_prof, stringIndexer_gend, stringIndexer_mar, vectorAssembler_features, rf, labelConverter])

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [52]:
train_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



In [53]:
model_rf = pipeline_rf.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [54]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)
print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.589319
Test Error = 0.410681


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist model

In this section you will learn how to store your pipeline and model in Watson Machine Learning repository by using python client libraries.

First, you must import client libraries.

**Note**: Apache® Spark 2.0 or higher is required.

In [55]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact

Authenticate to Watson Machine Learning service on Bluemix.

**Action**: Put authentication information from your instance of Watson Machine Learning service here.</div>

In [56]:
service_path = 'https://ibm-watson-ml.mybluemix.net'
instance_id = 'fc3f61c2-90a2-4903-a2af-9b83e250eb98'
username = '12e54e48-1a77-446c-b9e4-446436811a03'
password = '02cfd230-c454-4910-bb79-bc0058c1ee95'

**Tip**: service_path, user and password can be found on **Service Credentials** tab of service instance created in Bluemix. If you cannot see **instance_id** field in **Serice Credentials** generate new credentials by pressing **New credential (+)** button. 

In [57]:
ml_repository_client = MLRepositoryClient(service_path)
ml_repository_client.authorize("12e54e48-1a77-446c-b9e4-446436811a03", "02cfd230-c454-4910-bb79-bc0058c1ee95")

Create model artifact (abstraction layer).

In [58]:
model_artifact = MLRepositoryArtifact(model_rf, training_data=train_data, name="Product Line Prediction-3")

**Tip**: The MLRepositoryArtifact method expects a trained model object, training data, and a model name. (It is this model name that is displayed by the Watson Machine Learning service).

### 4.1: Save pipeline and model

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [59]:
saved_model = ml_repository_client.models.save(model_artifact)

Get saved model metadata from Watson Machine Learning.

**Tip**: Use *meta.available_props()* to get the list of available props.

In [60]:
saved_model.meta.available_props()

['inputDataSchema',
 'evaluationMetrics',
 'pipelineVersionHref',
 'modelVersionHref',
 'trainingDataRef',
 'pipelineType',
 'creationTime',
 'lastUpdated',
 'label',
 'authorEmail',
 'trainingDataSchema',
 'authorName',
 'version',
 'modelType',
 'runtime',
 'evaluationMethod']

In [61]:
print "modelType: " + saved_model.meta.prop("modelType")
print "trainingDataSchema: " + str(saved_model.meta.prop("trainingDataSchema"))
print "creationTime: " + str(saved_model.meta.prop("creationTime"))
print "modelVersionHref: " + saved_model.meta.prop("modelVersionHref")
print "label: " + saved_model.meta.prop("label")

modelType: sparkml-model-2.0
trainingDataSchema: {u'fields': [{u'nullable': True, u'type': u'string', u'name': u'PRODUCT_LINE', u'metadata': {}}, {u'nullable': True, u'type': u'string', u'name': u'GENDER', u'metadata': {}}, {u'nullable': True, u'type': u'integer', u'name': u'AGE', u'metadata': {}}, {u'nullable': True, u'type': u'string', u'name': u'MARITAL_STATUS', u'metadata': {}}, {u'nullable': True, u'type': u'string', u'name': u'PROFESSION', u'metadata': {}}], u'type': u'struct'}
creationTime: 2018-01-24 21:34:25.184000+00:00
modelVersionHref: https://ibm-watson-ml.mybluemix.net/v2/artifacts/models/7c8ce39b-aaaa-4b77-96e2-18c5052befe6/versions/4db13594-fa8d-4671-8823-58d61cfd6934
label: PRODUCT_LINE


**Tip**: **modelVersionHref** is our model unique indentifier in the Watson Machine Learning repository.

### 4.2: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [62]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)

You can print for example model name to make sure that model has been loaded correctly.

In [63]:
print str(loadedModelArtifact.name)

Product Line Prediction-3


As you can see the name is correct. You have already learned how save and load the model from Watson Machine Learning repository.

<a id="visualization"></a>
## 5. Predict locally and visualize

In this section you will learn how to score test data using loaded model and visualize the prediction results with plotly package.

### 5.1: Make local prediction using previously loaded model and test data

In this subsection you will score *predict_data* data set.

In [64]:
predictions = loadedModelArtifact.model_instance().transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [65]:
predictions.show(5)

+-----------------+------+---+--------------+----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|     PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|PROFESSION|label|PROFESSION_IX|GENDER_IX|MARITAL_STATUS_IX|          features|       rawPrediction|         probability|prediction|      predictedLabel|
+-----------------+------+---+--------------+----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|Camping Equipment|     F| 20|        Single|     Other|  0.0|          0.0|      1.0|              1.0|[1.0,20.0,1.0,0.0]|[5.54119162527029...|[0.27705958126351...|       1.0|Personal Accessories|
|Camping Equipment|     F| 20|        Single|     Other|  0.0|          0.0|      1.0|              1.0|[1.0,20.0,1.0,0.0]|[5.54119162527029...|[0.27705958126351...|       1.0|Personal Accessories|
|Camping E

By tabulating a count, you can see which product line is the most popular.

In [66]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show()

+--------------------+-----+
|      predictedLabel|count|
+--------------------+-----+
|   Camping Equipment|  717|
|      Golf Equipment|   61|
|Mountaineering Eq...|   75|
|Personal Accessories|  363|
+--------------------+-----+



### 5.2: Sample visualization of data with Plotly package

In this subsection you will explore prediction results with Plotly, which is an online analytics and data visualization tool.

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.<BR><BR>
!pip install plotly --user <BR>
!pip install cufflinks==0.8.2 --user

Import Plotly and other required packages.

In [28]:
! pip install cufflinks --upgrade
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go
init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

Requirement already up-to-date: cufflinks in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sd65-2f09ee29190a3a-560e341405d1/.local/lib/python2.7/site-packages
Requirement already up-to-date: plotly>=2.0.0 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sd65-2f09ee29190a3a-560e341405d1/.local/lib/python2.7/site-packages (from cufflinks)
Requirement already up-to-date: colorlover>=0.2 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sd65-2f09ee29190a3a-560e341405d1/.local/lib/python2.7/site-packages (from cufflinks)
Requirement already up-to-date: pandas in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sd65-2f09ee29190a3a-560e341405d1/.local/lib/python2.7/site-packages (from cufflinks)
Requirement already up-to-date: decorator>=4.0.6 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sd65-2f09ee29190a3a-560e341405d1/.local/lib/python2.7/site-packages (from plotly>=2.0.0->cufflinks)
Requirement already up-to-date: requests in /usr/local/src/bluemix_jupyter_bundle.v77/notebook/lib/python2.7

Convert the Apache Spark DataFrame to a Pandas DataFrame.

In [29]:
predictions_pdf = predictions.select("prediction", "predictedLabel", "GENDER", "AGE", "PROFESSION", "MARITAL_STATUS").toPandas()

Plot a pie chart that shows the predicted product-line interest.

In [30]:
cumulative_stats = predictions_pdf.groupby(['predictedLabel']).count()

product_data = [go.Pie(
            labels=cumulative_stats.index,
            values=cumulative_stats['GENDER'],
    )]

product_layout = go.Layout(
    title='Predicted product line client interest distribution',
)

fig = go.Figure(data=product_data, layout=product_layout)
iplot(fig)

With this data set, you might want to do some analysis of the mean AGE per product line by using a bar chart.

In [31]:
age_data = [go.Bar(
            y=predictions_pdf.groupby(['predictedLabel']).mean()["AGE"],
            x=cumulative_stats.index
            
    )]

age_layout = go.Layout(
    title='Mean AGE per predicted product line',
    xaxis=dict(
        title = "Product Line",
        showline=False,),
    yaxis=dict(
        title = "Mean AGE",
        ),
)

fig = go.Figure(data=age_data, layout=age_layout)
iplot(fig)

Based on bar plot you created, you might make the following conclusion: The mean age for clients that are interested in golf equipment is predicted to be over 50 years old.

<a id="scoring"></a>
## 6. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning REST API. 
For more information about REST APIs, see the [Swagger Documentation](http://watson-ml-api.mybluemix.net/).

To work with the Watson Machine Leraning REST API you must generate an access token. To do that you can use the following sample code:

In [32]:
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(username, password))
url = '{}/v3/identity/token'.format(service_path)
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

### 6.1: Create online scoring endpoint

Now you can create an online scoring endpoint. 

#### Get published_models url from instance details

In [33]:
endpoint_instance = service_path + "/v3/wml_instances/" + instance_id
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

response_get_instance = requests.get(endpoint_instance, headers=header)
print response_get_instance
print response_get_instance.text

<Response [200]>
{"metadata":{"guid":"517d09bc-d49b-4ba4-8ae3-1e1cdb3da51e","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/517d09bc-d49b-4ba4-8ae3-1e1cdb3da51e","created_at":"2017-12-15T15:48:09.056Z","modified_at":"2018-01-24T19:18:47.443Z"},"entity":{"source":"Bluemix","published_models":{"url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/517d09bc-d49b-4ba4-8ae3-1e1cdb3da51e/published_models"},"usage":{"expiration_date":"2018-02-01T00:00:00.000Z","computation_time":{"limit":18000,"current":6},"model_count":{"limit":200,"current":14},"prediction_count":{"limit":5000,"current":37},"deployment_count":{"limit":5,"current":3}},"plan_id":"3f6acf43-ede8-413a-ac69-f8af3bb0cbfe","status":"Active","organization_guid":"602c2643-5224-41fb-9c88-bf763a6bdd47","region":"us-south","account":{"id":"6abee94f116b995e21e4b952e2614e69","name":"Andrew Zhang's Account","type":"TRIAL"},"owner":{"ibm_id":"507NFW24AY","email":"andrew.zhang@ibm.com","user_id":"4ad23702-cd09-451e-a1c2-ddb9

In [34]:
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')
print endpoint_published_models

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/517d09bc-d49b-4ba4-8ae3-1e1cdb3da51e/published_models


Execute the following sample code that uses the published_models endpoint to get deployments url.

#### Get the list of published models

In [35]:
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

response_get = requests.get(endpoint_published_models, headers=header)
print response_get
print response_get.text

<Response [200]>
{"count":5,"resources":[{"metadata":{"guid":"3dde7a76-ad3e-4566-a91c-4009c35411b2","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/517d09bc-d49b-4ba4-8ae3-1e1cdb3da51e/published_models/3dde7a76-ad3e-4566-a91c-4009c35411b2","created_at":"2017-12-18T20:17:15.291Z","modified_at":"2018-01-08T16:55:05.016Z"},"entity":{"runtime_environment":"spark-2.0","learning_configuration_url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/517d09bc-d49b-4ba4-8ae3-1e1cdb3da51e/published_models/3dde7a76-ad3e-4566-a91c-4009c35411b2/learning_configuration","author":{"name":"Andrew Zhang","email":"andrew.zhang@ibm.com"},"name":"NLP Lab2","label_col":"is_duplicate","learning_iterations_url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/517d09bc-d49b-4ba4-8ae3-1e1cdb3da51e/published_models/3dde7a76-ad3e-4566-a91c-4009c35411b2/learning_iterations","training_data_schema":{"type":"struct","fields":[{"name":"id","type":"integer","nullable":true,"metadata":{"columnInfo":{"

#### Get published model deployment url

In [36]:
[endpoint_deployments] = [x.get('entity').get('deployments').get('url') for x in json.loads(response_get.text).get('resources') if x.get('metadata').get('guid') == saved_model.uid]

print endpoint_deployments

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/517d09bc-d49b-4ba4-8ae3-1e1cdb3da51e/published_models/403e3bad-557b-42b8-870c-bde2d644a390/deployments


#### Create online deployment for published model

In [37]:
payload_online = {"name": "Product Line Prediction", "description": "My Cool Deployment", "type": "online"}
response_online = requests.post(endpoint_deployments, json=payload_online, headers=header)

print response_online
print response_online.text

<Response [201]>
{"metadata":{"guid":"8c892c7d-a8ea-4a96-b263-d4a3ef71794b","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/517d09bc-d49b-4ba4-8ae3-1e1cdb3da51e/published_models/403e3bad-557b-42b8-870c-bde2d644a390/deployments/8c892c7d-a8ea-4a96-b263-d4a3ef71794b","created_at":"2018-01-24T19:19:06.800Z","modified_at":"2018-01-24T19:19:11.467Z"},"entity":{"runtime_environment":"spark-2.0","name":"Product Line Prediction","scoring_url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/517d09bc-d49b-4ba4-8ae3-1e1cdb3da51e/published_models/403e3bad-557b-42b8-870c-bde2d644a390/deployments/8c892c7d-a8ea-4a96-b263-d4a3ef71794b/online","description":"My Cool Deployment","published_model":{"author":{},"name":"Product Line Prediction-2","url":"https://ibm-watson-ml.mybluemix.net/v3/wml_instances/517d09bc-d49b-4ba4-8ae3-1e1cdb3da51e/published_models/403e3bad-557b-42b8-870c-bde2d644a390","guid":"403e3bad-557b-42b8-870c-bde2d644a390","created_at":"2018-01-24T19:19:06.766Z"},"model_t

In [38]:
scoring_url = json.loads(response_online.text).get('entity').get('scoring_url')
print scoring_url

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/517d09bc-d49b-4ba4-8ae3-1e1cdb3da51e/published_models/403e3bad-557b-42b8-870c-bde2d644a390/deployments/8c892c7d-a8ea-4a96-b263-d4a3ef71794b/online


Now, you can send (POST) new scoring records (new data) for which you would like to get predictions. To do that, execute the following sample code: 

In [39]:
payload_scoring = {"fields": ["GENDER","AGE","MARITAL_STATUS","PROFESSION"],"values": [["M",23,"Single","Student"],["M",55,"Single","Executive"]]}
response_scoring = requests.post(scoring_url, json=payload_scoring, headers=header)

print response_scoring.text

{
  "fields": ["GENDER", "AGE", "MARITAL_STATUS", "PROFESSION", "PRODUCT_LINE", "label", "PROFESSION_IX", "GENDER_IX", "MARITAL_STATUS_IX", "features", "rawPrediction", "probability", "prediction", "predictedLabel"],
  "values": [["M", 23, "Single", "Student", "Camping Equipment", 0.0, 6.0, 0.0, 1.0, [0.0, 23.0, 1.0, 6.0], [5.600716147152702, 6.482458372136143, 6.048004170730676, 0.20929155492307386, 1.6595297550574055], [0.2800358073576351, 0.32412291860680714, 0.3024002085365338, 0.010464577746153694, 0.08297648775287028], 1.0, "Personal Accessories"], ["M", 55, "Single", "Executive", "Camping Equipment", 0.0, 3.0, 0.0, 1.0, [0.0, 55.0, 1.0, 3.0], [6.227653744886563, 4.325198862164969, 8.031953760146816, 1.2319759269281225, 0.1832177058735289], [0.3113826872443282, 0.2162599431082485, 0.40159768800734086, 0.06159879634640614, 0.009160885293676447], 2.0, "Mountaineering Equipment"]]
}


As we can see we predict that a 23-year-old male student is interested in personal accessories (predictedLabel: Personal Accessories, prediction: 1.0). We can also see that single, 55 years old man is interested in mountaineering equipment.

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/pm_service_api_spark.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Automation Architect and Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.